In [1]:
import json
from collections import defaultdict
with open("data/train_500.txt", 'r') as f:
    lines = f.readlines()

entities = defaultdict(list)
samples = []
sample = []
texts = []
special_char = "<SPACE>"
text_entity_pair = []
entity_txt = ''
entity_name = ''
single_pair = defaultdict(list)
for i, line in enumerate(lines):
    line = line.strip()
    if len(line) == 0:
        if len(sample):
            samples.append(sample)
            text = "".join([t.split("\t")[0] for t in sample])
            text = text.replace(special_char, " ")
            texts.append(text)
            if entity_name:
                entities[entity_name].append("".join(entity_txt))
                single_pair[entity_name].append("".join(entity_txt))
            text_entity_pair.append(
                {"text": text, "entities": single_pair, "sample": sample})

        entity_txt = ''
        entity_name = ""
        single_pair = defaultdict(list)
        sample = []
        continue
    line = line.split(" ")

    if len(line) == 2:
        txt, tag = line
        sample.append("\t".join(line))
    elif len(line) == 1:
        tag = line[0]
        txt = " "
        sample.append("\t".join((special_char, line[0])))
    else:
        print(line)
    if tag == "O":
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
            single_pair[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_name = ""
    elif tag.startswith("B"):
        if entity_name:
            entities[entity_name].append("".join(entity_txt))
            single_pair[entity_name].append("".join(entity_txt))
        entity_txt = ''
        entity_txt += txt
        entity_name = tag.split("-")[-1]
    else:
        entity_txt += txt
        # entity_name = tag.split("-")[-1]

if len(sample):
    text = "".join([t.split("\t")[0] for t in sample])
    text = text.replace(special_char, " ")
    texts.append(text)
    samples.append(sample)
    if entity_name:
        entities[entity_name].append("".join(entity_txt))
        single_pair[entity_name].append("".join(entity_txt))
    text_entity_pair.append(
        {"text": text, "entities": single_pair, "sample": sample})



In [5]:
entities.keys()

dict_keys(['40', '4', '14', '5', '7', '11', '13', '8', '16', '29', '9', '12', '18', '1', '3', '22', '37', '39', '10', '36', '34', '31', '38', '54', '6', '30', '15', '2', '49', '21', '47', '23', '20', '50', '46', '41', '43', '48', '19', '52'])

In [4]:
from pypinyin import pinyin,Style

word = "中"
pinyin(word,style=Style.TONE3,heteronym=True)

[['。']]

In [4]:
from transformers.models.bert.tokenization_bert import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("/home/vocust001/pretrained_models/bert_rtb3")
text = "Bose SoundSport Free 真无线蓝牙耳机 运动耳机 博士防掉落耳塞 黑色"
# [t for t in text]

tokenizer.convert_tokens_to_ids(["a","b"])

[143, 144]

In [3]:
import torch.nn as nn
import torch

rnn = nn.GRU(10, 20, 2,batch_first=True)
input = torch.randn(30, 3, 10)
h0 = torch.randn(1, 3, 20)
output, hn = rnn(input)


In [35]:
hn.transpose(1,0).shape 

torch.Size([30, 2, 20])

In [15]:
embedding = nn.Embedding(7,32)
gru = nn.GRU(
            32,
            768,
            num_layers=1,
            batch_first=False,
            bidirectional=False,
        )
input = torch.LongTensor([[1,2,3,4],[2,3,1,5],[2,3,1,5]])
emb = embedding(input) 
# emb.shape 
output, hn = gru(emb)
for k,v in gru.state_dict().items():
    print(k,v.shape)

weight_ih_l0 torch.Size([2304, 32])
weight_hh_l0 torch.Size([2304, 768])
bias_ih_l0 torch.Size([2304])
bias_hh_l0 torch.Size([2304])


In [2]:
2304/768

3.0

In [28]:
from transformers import convert_tf_weight_name_to_pt_weight_name
convert_tf_weight_name_to_pt_weight_name("sk_emb/GRU/rnn/gru_cell/candidate/kernel")[0]

'GRU.rnn.gru_cell.candidate.weight'

In [6]:
from transformers import BertConfig,BertModel
model = BertModel.from_pretrained('/mnt/disk2/PythonProgram/NLPCode/PretrainModel/chinese_bert_base')
for k,v in model.state_dict().items():
    # if "sk_emb" in k and "adam_" not in k:
    print(k,v.shape)


Some weights of the model checkpoint at /mnt/disk2/PythonProgram/NLPCode/PretrainModel/chinese_bert_base were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


embeddings.position_ids torch.Size([1, 512])
embeddings.word_embeddings.weight torch.Size([21128, 768])
embeddings.position_embeddings.weight torch.Size([512, 768])
embeddings.token_type_embeddings.weight torch.Size([2, 768])
embeddings.LayerNorm.weight torch.Size([768])
embeddings.LayerNorm.bias torch.Size([768])
encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias torch.Size([768])
encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias torch.Size([768])
encoder.layer.0.attention.self.value.weight torch.Size([768, 768])
encoder.layer.0.attention.self.value.bias torch.Size([768])
encoder.layer.0.attention.output.dense.weight torch.Size([768, 768])
encoder.layer.0.attention.output.dense.bias torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.weight torch.Size([768])
encoder.layer.0.attention.output.LayerNorm.bias torch.Size([768])
encoder.layer.0.intermediate.dense.weight 

In [40]:
import re
layer_number = re.compile("_(\d+)")
layer_number.sub(r".\1","encoder.layer_6.attention.self.key.weight")

'encoder.layer.6.attention.self.key.weight'

In [36]:
import torch
model = torch.load("../data/plome.pt")
for k,v in model.items():
    if "adam_v" in k or "adam_m" in k:
        continue
    # print(convert_tf_weight_name_to_pt_weight_name(k,start_prefix_to_remove="bert")[0],v.shape)
    if "bert" in k:
        print(convert_tf_weight_name_to_pt_weight_name(k)[0],v.shape)

bert.pooler.dense.weight torch.Size([768, 768])
bert.encoder.layer_9.output.dense.weight torch.Size([768, 3072])
bert.encoder.layer_9.intermediate.dense.weight torch.Size([3072, 768])
bert.encoder.layer_9.intermediate.dense.bias torch.Size([3072])
bert.encoder.layer_9.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer_8.output.dense.weight torch.Size([768, 3072])
bert.encoder.layer_8.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer_8.intermediate.dense.bias torch.Size([3072])
bert.encoder.layer_8.attention.self.value.weight torch.Size([768, 768])
bert.encoder.layer_9.attention.self.key.bias torch.Size([768])
bert.encoder.layer_8.attention.output.dense.weight torch.Size([768, 768])
bert.encoder.layer_8.attention.output.LayerNorm.bias torch.Size([768])
bert.encoder.layer_7.output.dense.bias torch.Size([768])
bert.encoder.layer_7.intermediate.dense.weight torch.Size([3072, 768])
bert.encoder.layer_7.intermediate.dense.bias torch.Size([3072])
bert.encoder.lay

In [27]:
import torch
model1 = torch.load("../data/init.pt")
for k,v in model1.items():
    print(k,v.shape)

py_emb.pyemb.weight torch.Size([30, 32])
py_emb.GRU.weight_ih_l0 torch.Size([2304, 32])
py_emb.GRU.weight_hh_l0 torch.Size([2304, 768])
py_emb.GRU.bias_ih_l0 torch.Size([2304])
py_emb.GRU.bias_hh_l0 torch.Size([2304])
sk_emb.pyemb.weight torch.Size([7, 32])
sk_emb.GRU.weight_ih_l0 torch.Size([2304, 32])
sk_emb.GRU.weight_hh_l0 torch.Size([2304, 768])
sk_emb.GRU.bias_ih_l0 torch.Size([2304])
sk_emb.GRU.bias_hh_l0 torch.Size([2304])
embeddings.position_ids torch.Size([1, 512])
embeddings.word_embeddings.weight torch.Size([21128, 768])
embeddings.position_embeddings.weight torch.Size([512, 768])
embeddings.token_type_embeddings.weight torch.Size([2, 768])
embeddings.LayerNorm.weight torch.Size([768])
embeddings.LayerNorm.bias torch.Size([768])
encoder.layer.0.attention.self.query.weight torch.Size([768, 768])
encoder.layer.0.attention.self.query.bias torch.Size([768])
encoder.layer.0.attention.self.key.weight torch.Size([768, 768])
encoder.layer.0.attention.self.key.bias torch.Size([768])

In [7]:
from pypinyin import pinyin,lazy_pinyin,Style
lazy_pinyin("㐮",style=Style.TONE3)

['xiang1']

In [14]:
import torch
a = torch.tensor([[10,11,2,3,4,5,6],[10,11,2,3,4,5,6]])
mask = torch.tensor([[0,1,1,1,0,0,0],[0,1,1,1,1,1,0]]).type(torch.bool)
mask = ~mask.type(torch.bool)
torch.masked_fill(a,mask,torch.tensor(-100))
# mask

tensor([[-100,   11,    2,    3, -100, -100, -100],
        [-100,   11,    2,    3,    4,    5, -100]])

In [15]:
text = "山本小铁子\n\n山本小铁子，日本女性BL漫画家与小说插画家，现居大阪。出生于1月4日，摩羯座，A型血。左撇子，喜欢鸟类，家中有两只鹦鹉，此外亦热衷于观赏棒球赛事及收集相关物品，经常将个人兴趣和生活经验融入原创的漫画作品之中。\n\n笔名来自已故的日本职业摔角选手，又因为自己是女生的关系，故在后面加上一个「子」字。\n\n目前主要从事创作连载漫画、绘制小说插图与协助小说漫画化等工作，作品经常刊登于幻冬舍、、海王社、白泉社等日本知名出版社的杂志之中。\n\n原创作品风格温馨自然，内容有趣而诙谐，取材贴近生活，结局喜剧为主。不过做为BL漫画家，其作品的性爱场面明显偏少，用较多篇幅描绘主角间的暧昧情愫。\n\n起初是以少女漫画家的身分出道，在1995、1996年期间曾以「」为名，在白泉社杂志《花与梦》发表〈〉、〈〉、〈〉、〈〉、〈〉、〈〉等漫画作品，但过程并不顺利，以致于均未进一步集结成单行本。此外亦曾以「」为笔名。\n\n后来经朋友影响，接触到同人志贩售会与相关作品，转而成为同人志作家，也开始从事跟BL相关的二次创作，直至今日仍以「」与「」为社团名称参与同人活动，其作画的内容大多是参考日本偶像团体「Kinki Kids」以及动画「新机动战记钢弹W」中的角色衍生而来，近年则开始投入漫画《影子篮球员》的同人创作，并主打「黄黑」配对，现已推出《AS YOU LIKE IT》、《》、《first xxx》、《》、《I love you》、《Facta ,non verba》、《》等作品。\n\n2001年，受业界编辑邀约，开始尝试创作并发行BL向的连载漫画与单行本，持续至今。本身是业界少见的高产量作家，每个月平均要更新四部作品，完成一百页以上的创作，目前手上有九部作品正在连载，主要负责故事发想以及原画、上线的部分，其他部分是由助手田中小姐协助。\n\n过去除了同人志活动外很少公开露面，自2011年起，才开始配合部分单行本之新刊出版，在日本各地举办个人签名会。2013年8月15日，更接受台湾东贩之邀请来台湾（前一天已先到台湾），于第14届漫画博览会开幕当天举办签名会，作品《野蛮情人》第六册中文版亦同日发行，这是小铁子个人生涯第一次造访台湾。\n\n套色者为连载中、未完结的漫画。\n\n依他人之原著小说内容，漫画化后的非原创作品，剧情不限于BL题材。\n\n已有部分或全部单行本化的原创作品，均为BL向。\n\n在杂志上持续连载，尚未单行本化的原创作品。\n\n在杂志上曾经出现，但刊载一话后未持续连载，亦未单行本化的原创作品，均为BL向。\n\n做为相关纪念活动的赠品，剧情多为原创作品之番外，均为BL向。\n\n原创作品有声化后的专辑，均为BL向。\n\n于他人之原著小说作品中，协助封面、封底与内页部分之插画。\n\n一般向作品\n\n\nBL向作品\n\n\n早年除了同人志贩售场合外，很少公开露面，自2011年起，商业志签名会次数才明显增加。\n\n\n\n以下资讯多来自漫画后记中的自述。\n\n\n出版社\n\n编辑部网志\n\n线上免费阅览（正版）\n"
import re
sent_split = re.compile("[？？。！!]")

for sent in text.strip().split("\n"):
    sents = sent_split.split(sent)
    punct = sent_split.findall(sent)



In [18]:
sent = "山本小铁子，日本女性BL漫画家与小说插画家，现居大阪。出生于1月4日，摩羯座，A型血。左撇子，喜欢鸟类，家中有两只鹦鹉，此外亦热衷于观赏棒球赛事及收集相关物品，经常将个人兴趣和生活经验融入原创的漫画作品之中。"
sents = sent_split.split(sent)
punct = sent_split.findall(sent)
punct = punct+[""]
new_sents = []
for s,p in zip(*(sents,punct)):
    new_sents.append(s+p)
# print(sents)
# print(punct)
new_sents

['山本小铁子，日本女性BL漫画家与小说插画家，现居大阪。',
 '出生于1月4日，摩羯座，A型血。',
 '左撇子，喜欢鸟类，家中有两只鹦鹉，此外亦热衷于观赏棒球赛事及收集相关物品，经常将个人兴趣和生活经验融入原创的漫画作品之中。',
 '']